In [6]:
from gurobipy import *
import networkx as nx
import math
import re
import pandas as pd

In [7]:
def data_load(fileName):
    listSet = []
    list_temp = []
    start_flag = False
    with open(fileName, "r") as file:
        for line in file:
            if line.find('Days') > -1:
                days = int(line.split(":")[1])
            if line.find('Periods_per_day') > -1:
                periods = int(line.split(":")[1])
            if line.strip() == "":
                if list_temp:
                    listSet.append(list_temp.copy())
                    # print(list_temp)
                    list_temp.clear()
                start_flag = True
                continue
            if start_flag == True:
                if line.find(":") > -1:
                    continue
                else:
                    list_temp.append(line.split())

    df_courses = pd.DataFrame(listSet[0], columns=['courseID','teacher','countOfLectures','minWorkingDays','countOfStudents'])
    df_courses = df_courses.set_index('courseID', drop=True)
    df_courses[['countOfLectures']] = df_courses[['countOfLectures']].astype(int)
    df_courses[['minWorkingDays']] = df_courses[['minWorkingDays']].astype(int)
    df_courses[['countOfStudents']] = df_courses[['countOfStudents']].astype(int)

    df_rooms = pd.DataFrame(listSet[1], columns=['roomID', 'capacity'])
    df_rooms = df_rooms.set_index('roomID', drop=True)
    df_rooms[['capacity']] = df_rooms[['capacity']].astype(int)

    dict_curricula = {}
    df_unavailability_constraints = pd.DataFrame(listSet[3], columns=['courseID','day','dayPeriod'])
    df_unavailability_constraints = df_unavailability_constraints.set_index('courseID', drop=True)
    df_unavailability_constraints[['day']] = df_unavailability_constraints[['day']].astype(int)
    df_unavailability_constraints[['dayPeriod']] = df_unavailability_constraints[['dayPeriod']].astype(int)

    dict_temp = {}
    for item in listSet[2]:
        for i in range(len(item)):
            if i == 1:
                dict_temp['count'] = int(item[i])
            if i == 2:
                dict_temp['members'] = item[i:]
        dict_curricula[item[0]] = dict_temp.copy()
        dict_temp.clear()

    return df_courses, df_rooms, dict_curricula, df_unavailability_constraints, days, periods

In [8]:
def set2pairSet(init_set):
    result_list = []
    temp_list = list(init_set)
    for i in range(len(temp_list) - 1):
        for j in temp_list[i + 1:]:
            result_list.append({temp_list[i], j})
    return result_list

In [9]:
df_courses, df_rooms, dict_curricula, df_unavailability_constraints, days, periods = data_load('comp02.ctt')

# courses K
courses = df_courses.index.to_list()

# rooms R
rooms = df_rooms.index.to_list()

# curriculas Curriculas
curriculas = dict_curricula.keys()

# time_slots T
time_slots = []
for i in range(days):
    for j in range(periods):
        time_slots.append((i,j))

# teachers Teachers
teachers = df_courses['teacher'].unique()


courses_curriculas_pair = {}
for course in courses:
    for curricula in curriculas:
        if course in dict_curricula[curricula]['members']:
            # temp_list.append(curricula)
            if course not in courses_curriculas_pair:
                courses_curriculas_pair[course] = [curricula]
            else:
                courses_curriculas_pair[course].append(curricula)

In [10]:
courses_curriculas_pair



{'c0131': ['q000', 'q001', 'q002'],
 'c0211': ['q003'],
 'c0095': ['q028'],
 'c0152': ['q000', 'q001', 'q002'],
 'c0103': ['q029'],
 'c0111': ['q029'],
 'c0113': ['q029', 'q030', 'q031'],
 'c0115': ['q030', 'q031', 'q049', 'q050', 'q051', 'q057', 'q071', 'q072'],
 'c0119': ['q031'],
 'c0127': ['q028'],
 'c0007': ['q005'],
 'c0015': ['q007'],
 'c0019': ['q006', 'q007'],
 'c0020': ['q007'],
 'c0023': ['q004', 'q005', 'q009'],
 'c0080': ['q006'],
 'c0088': ['q006',
  'q010',
  'q036',
  'q037',
  'q038',
  'q039',
  'q040',
  'q041',
  'q042',
  'q043',
  'q044'],
 'c0044': ['q012'],
 'c0110': ['q019', 'q029'],
 'c0058': ['q013', 'q024'],
 'c0061': ['q014', 'q015', 'q016', 'q017', 'q045', 'q050'],
 'c0066': ['q014', 'q062', 'q063', 'q064'],
 'c0074': ['q015', 'q046', 'q047', 'q048'],
 'c0213': ['q003'],
 'c0219': ['q009'],
 'c0178': ['q018'],
 'c0186': ['q020', 'q021', 'q022', 'q025', 'q026', 'q027'],
 'c0193': ['q019', 'q065'],
 'c0108': ['q028'],
 'c0056': ['q012'],
 'c0195': ['q018'],
